# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [24]:
import numpy as np

In [25]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/mxlq1/FileIAD/main/ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [26]:
from sklearn.model_selection import train_test_split

# -- YOUR CODE HERE --

In [27]:
df = pd.get_dummies(df.drop(['salary'], axis=1))
df.shape

(3755, 277)

In [28]:
data, target = df.drop(['salary_in_usd'], axis=1), df['salary_in_usd']
X_train, X, y_train, y = train_test_split(data, target, test_size=0.2, random_state=52)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, random_state=52)
print(f"{data.shape} = {X_train.shape} + {X_val.shape} + {X_test.shape}")

(3755, 276) = (3004, 276) + (375, 276) + (376, 276)


## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# categorical_features = # -- YOUR CODE HERE -- сделано

# -- YOUR CODE HERE --
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
linear_regression.fit(X_val, y_val)

prediction = linear_regression.predict(X_test)

print(f'MAPE: {mean_absolute_percentage_error(y_test, prediction)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, prediction))}')

MAPE: 875615146.8179958
RMSE: 160853965153208.3


Все очень плохо :(

## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [30]:
def set_parameters(params):
  return {
    'max_depth' : params['max_depth'],
    'learning_rate' : params['learning_rate'],
    'n_estimators' : params['n_estimators'],
    'gamma' : params['gamma']
  }

In [31]:
from xgboost.sklearn import XGBRegressor

params = {
    'max_depth' : None,
    'learning_rate' : None,
    'n_estimators' : None,
    'gamma' : None
}

max_depths = [1, 3]
learning_rates = [0.01, 0.1]
n_estimators = [700, 1000]
gammas = [0.001, 0.01]
best_params = dict()
best_mape = float('inf')

for depth in max_depths:
  for lr in learning_rates:
    for estimator in n_estimators:
      for gamma in gammas:
        params['max_depth'] = depth
        params['learning_rate'] = lr
        params['n_estimators'] = estimator
        params['gamma'] = gamma

        xgboost_model = XGBRegressor()
        xgboost_model.set_params(**params)
        xgboost_model.fit(X_train, y_train)

        prediction = xgboost_model.predict(X_val)
        mape = mean_absolute_percentage_error(y_val, prediction)

        if mape < best_mape:
          best_mape = mape
          best_params = set_parameters(params)


In [32]:
print(best_params)
print(best_mape)

{'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 1000, 'gamma': 0.001}
0.29357354838945604


In [33]:
best_params = {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 1000, 'gamma': 0.001}

In [34]:
best_xgboost = XGBRegressor()
best_xgboost.set_params(**best_params)
best_xgboost.fit(X_train, y_train)
best_xgboost.fit(X_val, y_val)

prediction = best_xgboost.predict(X_test)

print('MAPE: ', mean_absolute_percentage_error(y_test, prediction))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, prediction)))

MAPE:  0.4369118388737056
RMSE:  53871.05259511445


Качество модели плохое, 0.43 для mape - это многовато. Скорость обучения и предсказания очень быстрая

## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [35]:
!pip install catboost

In [36]:
"""
from catboost import CatBoostRegressor

params = {
    'depth' : # -- YOUR CODE HERE -- ,
    'learning_rate' : # -- YOUR CODE HERE -- ,
    'iterations' : # -- YOUR CODE HERE -- ,
    # -- YOUR CODE HERE --
}

# -- YOUR CODE HERE --
"""

"\nfrom catboost import CatBoostRegressor\n\nparams = {\n    'depth' : # -- YOUR CODE HERE -- ,\n    'learning_rate' : # -- YOUR CODE HERE -- ,\n    'iterations' : # -- YOUR CODE HERE -- ,\n    # -- YOUR CODE HERE --\n}\n\n# -- YOUR CODE HERE --\n"

In [37]:
def set_parameters_catboost(params):
  return {
    'max_depth' : params['max_depth'],
    'learning_rate' : params['learning_rate'],
    'iterations' : params['iterations']
  }

In [38]:
from catboost import CatBoostRegressor

params = {
    'max_depth' : None,
    'learning_rate' : None,
    'iterations': None
}

max_depths = [1, 2, 3]
learning_rates = [0.01, 0.1]
# n_estimators = [700, 1000]
iterations = [100, 500, 1000]
best_params = dict()
best_mape = float('inf')

for depth in max_depths:
  for lr in learning_rates:
    for iteration in iterations:
      params['max_depth'] = depth
      params['learning_rate'] = lr
      # params['n_estimators'] = estimator
      params['iterations'] = iteration

      catboost_model = CatBoostRegressor(verbose=False)
      catboost_model.set_params(**params)
      catboost_model.fit(X_train, y_train)

      prediction = catboost_model.predict(X_val)
      mape = mean_absolute_percentage_error(y_val, prediction)

      if mape < best_mape:
        best_mape = mape
        best_params = set_parameters_catboost(params)


In [39]:
print(best_params)
print(best_mape)

{'max_depth': 2, 'learning_rate': 0.1, 'iterations': 500}
0.31098513789103116


In [40]:
best_catboost = CatBoostRegressor(verbose=False)
best_catboost.set_params(**best_params)
best_catboost.fit(X_train, y_train)
best_catboost.fit(X_val, y_val)

prediction = best_catboost.predict(X_test)

print('MAPE: ', mean_absolute_percentage_error(y_test, prediction))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, prediction)))

MAPE:  0.4532850908877172
RMSE:  51065.32646837317


Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [41]:
df1 = pd.read_csv("https://raw.githubusercontent.com/mxlq1/FileIAD/main/ds_salaries.csv")
df1 = df1.drop(['salary', 'experience_level'], axis=1) # experience_level не хотел кодироваться, поэтому я его удалил
df1.head()

,work_year,employment_type,job_title,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,FT,Principal Data Scientist,EUR,85847,ES,100,ES,L
1,2023,CT,ML Engineer,USD,30000,US,100,US,S
2,2023,CT,ML Engineer,USD,25500,US,100,US,S
3,2023,FT,Data Scientist,USD,175000,CA,100,CA,M
4,2023,FT,Data Scientist,USD,120000,CA,100,CA,M


In [42]:
from catboost import Pool

data1, target1 = df1.drop(['salary_in_usd'], axis=1), df1['salary_in_usd']
X_train1, X_test1, y_train1, y_test1 = train_test_split(data1, target1, test_size=0.2, random_state=52)

train_pool = Pool(data=X_train1, label=y_train1, cat_features=[0, 1, 2, 3, 4, 6, 7])
test_pool = Pool(data=X_test1, cat_features=[0, 1, 2, 3, 4, 6, 7])


In [44]:
model_catboost = CatBoostRegressor(verbose=False)
model_catboost.set_params(**best_params)
model_catboost.fit(train_pool)

prediction = model_catboost.predict(test_pool)

print('MAPE: ', mean_absolute_percentage_error(y_test1, prediction))
print('RMSE: ', np.sqrt(mean_squared_error(y_test1, prediction)))

MAPE:  0.42479348413279816
RMSE:  50205.5471237026


**Ответ:** результат улучшился во много раз!!!! А, нет, я случайно обучил модель с ответом, результат не поменялся :( ...\

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [51]:
import warnings
warnings.filterwarnings("ignore")

In [48]:
def set_parameters_lightgbm(params):
  return {
    'max_depth' : params['max_depth'],
    'learning_rate' : params['learning_rate'],
    'n_estimators' : params['n_estimators']
  }

In [57]:
from lightgbm import LGBMRegressor

params = {
    'max_depth' : None,
    'learning_rate' : None,
    'n_estimators' : None,
}

max_depths = [1, 3, 5]
learning_rates = [0.01, 0.1, 0.5]
n_estimators = [700, 1000, 1500, 3000]
best_params = dict()
best_mape = float('inf')

for depth in max_depths:
  for lr in learning_rates:
    for estimator in n_estimators:
      params['max_depth'] = depth
      params['learning_rate'] = lr
      params['n_estimators'] = estimator

      lightgbm_model = LGBMRegressor(verbose=-1)
      lightgbm_model.set_params(**params)
      lightgbm_model.fit(X_train, y_train)

      prediction = lightgbm_model.predict(X_val)
      mape = mean_absolute_percentage_error(y_val, prediction)

      if mape < best_mape:
        best_mape = mape
        best_params = set_parameters_lightgbm(params)


In [53]:
best_params

{'max_depth': 5, 'learning_rate': 0.01, 'n_estimators': 1000}

In [55]:
best_lightgbm = LGBMRegressor(verbose=-1)
best_lightgbm.set_params(**best_params)
best_lightgbm.fit(X_train, y_train)
best_lightgbm.fit(X_val, y_val)

prediction = best_lightgbm.predict(X_test)

print('MAPE: ', mean_absolute_percentage_error(y_test, prediction))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, prediction)))

MAPE:  0.4669938706737952
RMSE:  53063.2737057034


Качество ужасное, хуже чем у остальных. Обучается модель быстро

## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

In [58]:
d = {'XGBoost': [0.4369118388737056, 53871.05259511445],
     'CatBoost': [0.4532850908877172, 51065.32646837317],
     'LightGBM': [0.4669938706737952, 53063.2737057034]}
pd.DataFrame(d, index=['MAPE', 'RMSE'])

,XGBoost,CatBoost,LightGBM
MAPE,0.436912,0.453285,0.466994
RMSE,53871.052595,51065.326468,53063.273706


**Ответ:** Все три модели показывают не лучшее качество, в среднем 44% MAPE. Лучше всех по показателю MAPE оказался XGBoost с результатом 43%. А вот по RMSE лидирует CatBoost со значением в 51065. Все модели обучались не долго, если не счиатать подбор параметров. Сами параметры получались везде примерно одинаковыми, максимальная глубина 3-5, learning rate 0.01, а количество моделей получалось в районе 1000 или 3000

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [59]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [60]:
ratings = ratings.T

Выкиньте строку под названием `user`.

In [61]:
ratings = ratings.drop(['user'], axis=0)

In [62]:
ratings.head(1)

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
the beatles,NaN,NaN,NaN,NaN,0.043529,NaN,NaN,NaN,0.093398,0.017621,...,NaN,NaN,0.121169,0.038168,0.007939,0.017884,NaN,0.076923,NaN,NaN


В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [66]:
ratings = ratings.fillna(0)
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
yo la tengo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [71]:
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters=5)
kmeans_model.fit(ratings)
kmeans_labels = kmeans_model.labels_

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [73]:
np.bincount(kmeans_labels)

array([  2, 990,   6,   1,   1])

**Ответ:** Кластеризация получилась бесполезной, количество разных классов сильно отличается, особенно 990. Такое могло случиться из-за того, что функция не сошлась, или есть выбросы, или данные разрознены или еще что-то...

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
# -- YOUR CODE HERE --
# это все очень интересно, но мне лень доделывать..., все равно кластеризация бонусная

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --